### The following data was acquired from a course provided database built on the OMOP model with Synthea data
A number of different tables were created and then exported from the database for use.  
https://synthetichealth.github.io/synthea/

#### Isolating COVID-19 Related Conditions
In this step, I created a temporary table, `covid_occurrences`, to extract and isolate all occurrences of COVID-19-related conditions based on a predefined list of `condition_concept_id`'s provided during classwork. This allows for a more focused analysis of comorbidities and treatment outcomes related to COVID-19. This table will be filtered to only contain the first diagnosis (by date) for each included person_id so that covid comorbidities can be studied in the context of a first covid infection.

##### SQL Code:
```sql
create temp table covid_occurrences as
with covid_condition_concepts as (
    select unnest(array[
        3661631, 37310254, 756039, 37310284, 3656667, 3661405, 3661748,
        3661632, 3656668, 3661408, 3663281, 37310283, 3661885, 3661406,
        37310287, 3655977, 756031, 3662381, 37310286, 3655975, 3656669,
        3655976, 37311061
    ]) as condition_concept_id
),
earliest_covid_occurrences as (
    -- Step 1: Find the earliest start date for each person_id
    select 
        co.person_id,
        min(co.condition_start_date) as earliest_start_date
    from condition_occurrence co
    join covid_condition_concepts ccc on co.condition_concept_id = ccc.condition_concept_id
    group by co.person_id
)
-- Step 2: Select only one row for each person_id with the earliest condition_start_date
select distinct on (eco.person_id)
    eco.person_id,
    eco.earliest_start_date as condition_start_date,
    co.condition_end_date
from earliest_covid_occurrences eco
join condition_occurrence co on eco.person_id = co.person_id
    and eco.earliest_start_date = co.condition_start_date
order by eco.person_id, co.condition_end_date nulls last;


```

#### Step: Creating the `covid_person` Temporary Table
Here we aqcuired records of all people in the `person` table who had a `person_id` corresponding to a diagnosis of COVID-19. Certain descriptive fields were acquired for their potential to be of use in analysis.

##### SQL Code:
```sql
create temp table covid_person as
select
    co.person_id,
    p.gender_source_value,
    p.birth_datetime,
    date_part('year', age(co.condition_start_date, p.birth_datetime)) as age_at_covid,  -- Calculated age at diagnosis
    p.race_source_value,
    p.ethnicity_source_value,
    co.condition_start_date as covid_start_date
from covid_occurrences co
join person p on co.person_id = p.person_id;

#### Step: Creating the `covid_comorbidities` Temporary Table

In this step, I created a temporary table named `covid_comorbidities` to capture comorbid conditions for patients diagnosed with COVID-19. These conditions were filtered down to only include conditions that were ongoing at the time of COVID-19 diagnosis. This helps in identifying pre-existing conditions that could impact the severity or outcomes of COVID-19. Potentially, it would be worth filtering further to only include conditions that started 'x' months before covid diagnosis depending on application.

##### SQL Code:
```sql
create temp table covid_comorbidities as
select
    co.person_id,
    cco.condition_concept_id,
    c.concept_name,
    cco.condition_start_date,   
    cco.condition_end_date      
from covid_occurrences co
join condition_occurrence cco on co.person_id = cco.person_id
join concept c on cco.condition_concept_id = c.concept_id
where (cco.condition_end_date is null 
       or cco.condition_end_date > co.condition_start_date)  -- ongoing condition or ended after COVID start
  and cco.condition_start_date < co.condition_start_date      -- started before COVID diagnosis
order by co.person_id, cco.condition_start_date;



#### Step: Creating the `covid_deaths` Temporary Table

This table includes individuals who have had a recorded occurrence of COVID-19 and have also died. It does not necessarily indicate that they died of COVID-19; it simply captures those who had a COVID-19 diagnosis at some point and subsequently passed away. We are also addressing known duplicate entries in the `death` table.

##### SQL Code:
```sql
create temp table covid_deaths as
select 
    d.person_id,
    d.death_datetime,
    d.cause_concept_id,
    c.concept_name as cause_of_death 
from covid_person cp
join death d on cp.person_id = d.person_id
left join concept c on d.cause_concept_id = c.concept_id;  -- Ensuring it's a LEFT join in case some deaths don't have a cause




### The following two tables were acquired but not yet used in analysis.
They contain more records on medication and procedure history at the time of COVID treatment. These could aid more in-depth analysis of COVID outcomes.

#### Step: Creating the `covid_drug_exposure` Temporary Table

This table captures the drug treatments administered to COVID-19 patients during the period of their recorded COVID-19 diagnosis. It links the `drug_exposure` table with the `covid_occurrences` table to identify which medications were given and their respective durations.

##### SQL Code:
```sql
create temp table covid_drug_exposure as
select 
    co.person_id, 
    de.drug_concept_id, 
    de.drug_exposure_start_date, 
    de.drug_exposure_end_date, 
    dc.concept_name as drug_name
from covid_occurrences co
join drug_exposure de on co.person_id = de.person_id
join concept dc on de.drug_concept_id = dc.concept_id
where de.drug_exposure_start_date between co.condition_start_date and co.condition_end_date;


#### Step: Creating the `covid_procedures` Temporary Table

This table captures the medical procedures performed on COVID-19 patients during the period of their recorded COVID-19 diagnosis. It links the `procedure_occurrence` table with the `covid_occurrences` table to identify which procedures were conducted and when they occurred.

##### SQL Code:
```sql
create temp table covid_procedures as
select 
    co.person_id,
    po.procedure_concept_id,
    pc.concept_name as procedure_name,
    po.procedure_date
from covid_occurrences co
join procedure_occurrence po on co.person_id = po.person_id
join concept pc on po.procedure_concept_id = pc.concept_id
where po.procedure_date between co.condition_start_date and co.condition_end_date
order by co.person_id, po.procedure_date;


#### Exporting to .csv files  

The tables were then exported as .csv files and can be found in the adjoining `data` folder in this repository.

- `covid_comorbidities.csv`
- `covid_occurrences.csv`
- `covid_deaths.csv`
- `covid_drug_exposures.csv`
- `covid_procedures.csv`
